In [ ]:
%pip install openpyxl nbformat plotly widgetsnbextension~=4.0.10 ipywidgets==8.1.2

In [ ]:
import numpy as np
from scipy.integrate import cumulative_trapezoid
from scipy.signal import find_peaks
from scipy import signal
import plotly.express as px
import plotly.graph_objects as go
from dash import Dash, dcc, html, Input, Output, callback
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from ipywidgets import widgets
from dataclasses import dataclass
from typing import TextIO
import io
import re

In [ ]:
def load_dataframe(file: bytes, debug=False):
    df_raw = pd.read_csv(io.BytesIO(file), delimiter='\t', names=['t', 'flow', 'fo2', 'fco2'])
    first_non_zero_flow_row = 0
    flow_thresh = 0.2
    flow_almost_zero = df_raw[df_raw['flow'].abs() > flow_thresh]
    if len(flow_almost_zero) > 0:
        # start a couple of samples before flow takes >0 values
        flow_almost_zero_cutoff = max(0, flow_almost_zero.index[0] - 300) 
        if flow_almost_zero_cutoff > 0:
            if debug: print(f'Dropping first {flow_almost_zero_cutoff} rows with flow<{flow_thresh}')
            df_raw.drop(range(flow_almost_zero_cutoff), inplace=True)
    df_raw.reset_index(inplace=True, drop=True)
    df_raw['t'] = df_raw['t'] - df_raw.loc[0, 't']

    sampling_freq_hz = 1 / (df_raw['t'][:-1] - df_raw['t'].shift(1)[1:]).mean()
    filter_freq_hz = 3.15*1e-2
    df_raw['instant_vol_raw'] = cumulative_trapezoid(y=df_raw['flow'], x=df_raw['t'], initial=0) 
    sos = signal.butter(4, Wn=filter_freq_hz / sampling_freq_hz, btype='highpass', output='sos')
    flow_filtered = signal.sosfilt(sos, df_raw['flow'])
    df_raw['instant_vol'] = cumulative_trapezoid(y=flow_filtered, x=df_raw['t'], initial=0) 
    if debug: print(f'Sum over all instantaneous volume: {df_raw.instant_vol.sum()}')

    return df_raw


In [ ]:
def on_analyze_clicked(file: bytes, output, image_config={'scale': 1}, debug=False):
    global df_raw
    df_raw = load_dataframe(file, debug)
    df_raw['ts'] = pd.to_datetime(df_raw['t'], unit='s')
    fig = px.line(df_raw, y=['instant_vol', 'flow'], x='ts', height=500)
    names={'flow': 'Flow', 'instant_vol': 'Vol. Instant.'}
    fig.for_each_trace(lambda t: t.update(name = names[t.name],
                                      legendgroup = names[t.name]
                                         )
                  )
    config = {
        'toImageButtonOptions': {
            'format': 'png', # one of png, svg, jpeg, webp
            #'filename': 'custom_image',
            **image_config
        }
    }
    fig.show(config=config)

In [ ]:
select_display = widgets.Output()
plot_display = widgets.Output()
select = None
width_input = None
height_input = None
scale_input = None
file = None
def f():
    global file
    if not select:
        return
    file = select.value
    plot_display.clear_output()
    with plot_display:
        image_config = { 'scale': scale_input.value }
        if width_input.value != 0: image_config['width'] = width_input.value
        if height_input.value != 0: image_config['height'] = height_input.value
        on_analyze_clicked(select.value, plot_display, image_config=image_config)

def on_upload_changed(inputs):
    global select
    global width_input
    global height_input
    global scale_input
    with select_display:
        select_display.clear_output()
        files = [f for f in inputs['new'] if f.name.endswith('raw.log')]
        select = widgets.Select(options=[(str(f.name), f.content) for f in files])
        button = widgets.Button(description='Analyze')
        button.on_click(lambda button: f())
        display(select)
        display(widgets.Label(value="Screenshot settings:"))
        style = {'description_width': 'initial'}
        width_input = widgets.IntText(value=0, description='width (0 = default)', style=style)
        height_input = widgets.IntText(value=0, description='height (0 = default)', style=style)
        scale_input = widgets.FloatText(value=1, description='scale')
        display(width_input, height_input, scale_input)
        display(button)

upload = widgets.FileUpload(
    multiple=True
)

upload.observe(on_upload_changed, names='value')

In [ ]:
upload

In [ ]:
display(select_display)
display(plot_display)